# **Trabalho Prático: Escalonamento e Sincronismo -  C012**

#### Equipe: Iza Lopes Ribeiro e Wiliane Carolina Silva

🏁 Imagine as modalidades olímpicas como 'corridas' em um computador. Para gerenciar essas 'corridas', usaremos dois juízes: o FCFS, que dá preferência para quem chega primeiro, e o Round Robin, que alterna entre os competidores a cada intervalo de tempo. Vamos cronometrar cada juiz para descobrir qual deles organiza as 'corridas' de forma mais eficiente, considerando o tempo total da competição, o tempo de espera de cada modalidade e o uso dos recursos do computador.

### Simulação das Modalidades e Escalonamento FCFS

In [2]:
import threading
import time
import random
from sports import sports

# Função para simular a execução do esporte
def sport(title, duration, daily_total_hours, lock):
    start_time = time.perf_counter()  # Captura o tempo inicial
    print(f'Modalidade {title} iniciada - tempo inicio: {start_time:.3f}')

    time.sleep(duration)  # Simula a execução do esporte pelo tempo de duração

    end_time = time.perf_counter()  # Captura o tempo final
    print(f'Modalidade {title} finalizada - tempo final: {end_time:.3f}')

    with lock:
        daily_total_hours[0] += duration

    print(f'Modalidade {title} concluída após {duration:.3f} horas. '
          f'Espera: {end_time - start_time:.3f} segundos.')

# Escalonamento FCFS
def escalonamento_fcfs(pending_sports, daily_total_hours, lock):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport, args=(title, duration, daily_total_hours, lock))
        executed_today.append((t, title))
        t.start()
        t.join()  # FCFS não permite interrupção, espera até terminar antes de iniciar o próximo

    return executed_today

# Simulação dos Jogos Olímpicos com FCFS
def start_olympics_fcfs():
    print('\nOs Jogos Olímpicos com FCFS foram iniciados!\n')

    day_duration = 12  # Duração de cada dia em horas
    day_count = 0  # Contador de dias
    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior

    lock = threading.Lock()

    while all_sports or pending_sports:
        day_count += 1
        print(f'--- Início do dia {day_count} ---\n')

        day_remaining = day_duration  
        daily_total_hours = [0]

        # Adiciona esportes ao dia conforme há tempo disponível
        while all_sports and day_remaining > 0:
            title = all_sports.pop(0)
            duration = random.uniform((1/60), 3.5)  # Gera um tempo aleatório entre 1 minuto e 5 horas

            if duration <= day_remaining:
                pending_sports.append((title, duration))
                day_remaining -= duration
            else:
                pending_sports.append((title, duration))
                break

        executed_today = escalonamento_fcfs(pending_sports, daily_total_hours, lock)
        
        executed_sports = ', '.join([title for _, title in executed_today])
        print(f'\n--- Fim do dia {day_count} ---\nEsportes executados: {executed_sports}\n')
        print(f'Total de horas de esportes executados no dia {day_count}: {daily_total_hours[0]:.3f} horas\n')
        
    print(f'Os Jogos Olímpicos foram finalizados após {day_count} dias!\n')

# Inicia a simulação com FCFS
start_olympics_fcfs()



Os Jogos Olímpicos com FCFS foram iniciados!

--- Início do dia 1 ---

Modalidade Atletismo iniciada - tempo inicio: 294853.970
Modalidade Atletismo finalizada - tempo final: 294854.358
Modalidade Atletismo concluída após 0.373 horas. Espera: 0.388 segundos.
Modalidade Badminton iniciada - tempo inicio: 294854.361
Modalidade Badminton finalizada - tempo final: 294855.280
Modalidade Badminton concluída após 0.912 horas. Espera: 0.918 segundos.
Modalidade Basquete iniciada - tempo inicio: 294855.284
Modalidade Basquete finalizada - tempo final: 294858.458
Modalidade Basquete concluída após 3.163 horas. Espera: 3.173 segundos.
Modalidade Basquete 3x3 iniciada - tempo inicio: 294858.462
Modalidade Basquete 3x3 finalizada - tempo final: 294860.371
Modalidade Basquete 3x3 concluída após 1.908 horas. Espera: 1.909 segundos.
Modalidade Boxe iniciada - tempo inicio: 294860.375
Modalidade Boxe finalizada - tempo final: 294860.967
Modalidade Boxe concluída após 0.584 horas. Espera: 0.591 segundo

### Escalonamento Round Robin (RR)

In [2]:
# Escalonamento Round Robin
def escalonamento_rr(pending_sports, daily_total_hours, lock, quantum=2):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport, args=(title, min(duration, quantum), daily_total_hours, lock))
        executed_today.append((t, title))

        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

    return executed_today

# Simulação dos Jogos Olímpicos com Round Robin
def start_olympics_rr():
    print('\nOs Jogos Olímpicos com Round Robin foram iniciados!\n')

    day_duration = 12  # Duração de cada dia em horas
    day_count = 0  # Contador de dias
    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior

    lock = threading.Lock()

    while all_sports or pending_sports:
        day_count += 1
        print(f'--- Início do dia {day_count} ---\n')

        day_remaining = day_duration  
        daily_total_hours = [0]

        # Adiciona esportes ao dia conforme há tempo disponível
        while all_sports and day_remaining > 0:
            title = all_sports.pop(0)
            duration = random.uniform((1/60), 5)  # Gera um tempo aleatório entre 1 minuto e 5 horas

            if duration <= day_remaining:
                pending_sports.append((title, duration))
                day_remaining -= duration
            else:
                pending_sports.append((title, duration))
                break

        executed_today = escalonamento_rr(pending_sports, daily_total_hours, lock)
        
        executed_sports = ', '.join([title for _, title in executed_today])
        print(f'\n--- Fim do dia {day_count} ---\nEsportes executados: {executed_sports}\n')
        print(f'Total de horas de esportes executados no dia {day_count}: {daily_total_hours[0]:.3f} horas\n')

    print(f'Os Jogos Olímpicos foram finalizados após {day_count} dias!\n')

# Inicia a simulação com Round Robin
start_olympics_rr()


Os Jogos Olímpicos com Round Robin foram iniciados!

--- Início do dia 1 ---

Modalidade Atletismo iniciada - tempo inicio: 267532.401
Modalidade Atletismo finalizada - tempo final: 267534.150
Modalidade Atletismo concluída após 1.744 horas. Espera: 1.749 segundos.
Modalidade Badminton iniciada - tempo inicio: 267534.168
Modalidade Badminton finalizada - tempo final: 267536.170
Modalidade Badminton concluída após 2.000 horas. Espera: 2.003 segundos.
Modalidade Basquete iniciada - tempo inicio: 267536.173
Modalidade Basquete finalizada - tempo final: 267536.771
Modalidade Basquete concluída após 0.591 horas. Espera: 0.598 segundos.
Modalidade Basquete 3x3 iniciada - tempo inicio: 267536.774
Modalidade Basquete 3x3 finalizada - tempo final: 267537.799
Modalidade Basquete 3x3 concluída após 1.017 horas. Espera: 1.026 segundos.
Modalidade Boxe iniciada - tempo inicio: 267537.803
Modalidade Boxe finalizada - tempo final: 267539.812
Modalidade Boxe concluída após 2.000 horas. Espera: 2.009 

**🪢 Ao sincronizar o acesso das threads aos recursos compartilhados, garantiremos uma experiência mais fluida e confiável para o usuário. Essa medida evitará travamentos e erros causados por acessos simultâneos não controlados.**

### Implementação dos Mecanismos de Sincronização

####  Semáforo Binário

In [7]:
binary_semaphore = threading.Semaphore(1)

def sport_with_binary_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo Binário] Modalidade {title} aguardando acesso.')
    with binary_semaphore:
        print(f'[Semáforo Binário] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Semáforo Binário] Modalidade {title} liberou o acesso.')


#### Monitor

In [8]:
monitor_lock = threading.Lock()

def sport_with_monitor(title, duration, daily_total_hours):
    print(f'[Monitor] Modalidade {title} aguardando acesso.')
    with monitor_lock:
        print(f'[Monitor] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Monitor] Modalidade {title} liberou o acesso.')


#### Semáforo de Contagem

In [9]:
counting_semaphore = threading.BoundedSemaphore(2)  # Permite até 2 threads simultâneas

def sport_with_counting_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo de Contagem] Modalidade {title} aguardando acesso.')
    with counting_semaphore:
        print(f'[Semáforo de Contagem] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Semáforo de Contagem] Modalidade {title} liberou o acesso.')


### Escalonamento com Sincronismo

#### FCFS + Semáforo Binário

In [10]:
import threading
import time
import random
from sports import sports

# Função original de execução do esporte, agora com semáforo binário
binary_semaphore = threading.Semaphore(1)

def sport_with_binary_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo Binário] Modalidade {title} aguardando acesso.')
    with binary_semaphore:
        print(f'[Semáforo Binário] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Semáforo Binário] Modalidade {title} liberou o acesso.')

# Escalonamento FCFS com semáforo binário
def escalonamento_fcfs(pending_sports, daily_total_hours, lock):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport_with_binary_semaphore, args=(title, duration, daily_total_hours))
        executed_today.append((t, title))
        t.start()
        t.join()  # FCFS não permite interrupção, espera até terminar antes de iniciar o próximo

    return executed_today

# Simulação dos Jogos Olímpicos com FCFS e semáforo binário
def start_olympics_fcfs():
    print('\nOs Jogos Olímpicos com FCFS e semáforo binário foram iniciados!\n')

    day_duration = 12  # Duração de cada dia em horas
    day_count = 0  # Contador de dias
    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior

    lock = threading.Lock()

    while all_sports or pending_sports:
        day_count += 1
        print(f'--- Início do dia {day_count} ---\n')

        day_remaining = day_duration  
        daily_total_hours = [0]

        # Adiciona esportes ao dia conforme há tempo disponível
        while all_sports and day_remaining > 0:
            title = all_sports.pop(0)
            duration = random.uniform((1/60), 5)  # Gera um tempo aleatório entre 1 minuto e 5 horas

            if duration <= day_remaining:
                pending_sports.append((title, duration))
                day_remaining -= duration
            else:
                pending_sports.append((title, duration))
                break

        executed_today = escalonamento_fcfs(pending_sports, daily_total_hours, lock)
        
        executed_sports = ', '.join([title for _, title in executed_today])
        print(f'\n--- Fim do dia {day_count} ---\nEsportes executados: {executed_sports}\n')
        print(f'Total de horas de esportes executados no dia {day_count}: {daily_total_hours[0]:.3f} horas\n')
        
    print(f'Os Jogos Olímpicos foram finalizados após {day_count} dias!\n')

# Inicia a simulação com FCFS e semáforo binário
start_olympics_fcfs()



Os Jogos Olímpicos com FCFS e semáforo binário foram iniciados!

--- Início do dia 1 ---

[Semáforo Binário] Modalidade Atletismo aguardando acesso.
[Semáforo Binário] Modalidade Atletismo iniciou execução.
Modalidade Atletismo iniciada - tempo inicio: 268090.121
Modalidade Atletismo finalizada - tempo final: 268090.186
Modalidade Atletismo concluída após 0.063 horas. Espera: 0.065 segundos.
[Semáforo Binário] Modalidade Atletismo liberou o acesso.
[Semáforo Binário] Modalidade Badminton aguardando acesso.
[Semáforo Binário] Modalidade Badminton iniciou execução.
Modalidade Badminton iniciada - tempo inicio: 268090.187
Modalidade Badminton finalizada - tempo final: 268092.966
Modalidade Badminton concluída após 2.778 horas. Espera: 2.779 segundos.
[Semáforo Binário] Modalidade Badminton liberou o acesso.
[Semáforo Binário] Modalidade Basquete aguardando acesso.
[Semáforo Binário] Modalidade Basquete iniciou execução.
Modalidade Basquete iniciada - tempo inicio: 268092.970
Modalidade B

### RR + Semáforo de Contagem

In [11]:
import threading
import time
import random
from sports import sports

# Função original de execução do esporte, agora com semáforo de contagem
counting_semaphore = threading.BoundedSemaphore(2)  # Permite até 2 threads simultâneas

def sport_with_counting_semaphore(title, duration, daily_total_hours):
    print(f'[Semáforo de Contagem] Modalidade {title} aguardando acesso.')
    with counting_semaphore:
        print(f'[Semáforo de Contagem] Modalidade {title} iniciou execução.')
        sport(title, duration, daily_total_hours, threading.Lock())
    print(f'[Semáforo de Contagem] Modalidade {title} liberou o acesso.')

# Escalonamento Round Robin com semáforo de contagem
def escalonamento_rr(pending_sports, daily_total_hours, lock, quantum=2):
    executed_today = []
    while pending_sports:
        title, duration = pending_sports.pop(0)
        t = threading.Thread(target=sport_with_counting_semaphore, args=(title, min(duration, quantum), daily_total_hours))
        executed_today.append((t, title))

        t.start()
        t.join()

        # Se o esporte não terminou, retorna com o tempo restante
        if duration > quantum:
            pending_sports.append((title, duration - quantum))

    return executed_today

# Simulação dos Jogos Olímpicos com Round Robin e semáforo de contagem
def start_olympics_rr():
    print('\nOs Jogos Olímpicos com Round Robin e semáforo de contagem foram iniciados!\n')

    day_duration = 12  # Duração de cada dia em horas
    day_count = 0  # Contador de dias
    all_sports = sports.copy()  # Copia dos esportes disponíveis
    pending_sports = []  # Lista de esportes que não puderam ser executados no dia anterior

    lock = threading.Lock()

    while all_sports or pending_sports:
        day_count += 1
        print(f'--- Início do dia {day_count} ---\n')

        day_remaining = day_duration  
        daily_total_hours = [0]

        # Adiciona esportes ao dia conforme há tempo disponível
        while all_sports and day_remaining > 0:
            title = all_sports.pop(0)
            duration = random.uniform((1/60), 5)  # Gera um tempo aleatório entre 1 minuto e 5 horas

            if duration <= day_remaining:
                pending_sports.append((title, duration))
                day_remaining -= duration
            else:
                pending_sports.append((title, duration))
                break

        executed_today = escalonamento_rr(pending_sports, daily_total_hours, lock)
        
        executed_sports = ', '.join([title for _, title in executed_today])
        print(f'\n--- Fim do dia {day_count} ---\nEsportes executados: {executed_sports}\n')
        print(f'Total de horas de esportes executados no dia {day_count}: {daily_total_hours[0]:.3f} horas\n')

    print(f'Os Jogos Olímpicos foram finalizados após {day_count} dias!\n')

# Inicia a simulação com Round Robin e semáforo de contagem
start_olympics_rr()



Os Jogos Olímpicos com Round Robin e semáforo de contagem foram iniciados!

--- Início do dia 1 ---

[Semáforo de Contagem] Modalidade Atletismo aguardando acesso.
[Semáforo de Contagem] Modalidade Atletismo iniciou execução.
Modalidade Atletismo iniciada - tempo inicio: 268192.272
Modalidade Atletismo finalizada - tempo final: 268194.280
Modalidade Atletismo concluída após 2.000 horas. Espera: 2.008 segundos.
[Semáforo de Contagem] Modalidade Atletismo liberou o acesso.
[Semáforo de Contagem] Modalidade Badminton aguardando acesso.
[Semáforo de Contagem] Modalidade Badminton iniciou execução.
Modalidade Badminton iniciada - tempo inicio: 268194.298
Modalidade Badminton finalizada - tempo final: 268196.306
Modalidade Badminton concluída após 2.000 horas. Espera: 2.008 segundos.
[Semáforo de Contagem] Modalidade Badminton liberou o acesso.
[Semáforo de Contagem] Modalidade Basquete aguardando acesso.
[Semáforo de Contagem] Modalidade Basquete iniciou execução.
Modalidade Basquete inici

#### Comparação de resultado: 

FCFS será mais simples, com as modalidades executando uma por vez. O tempo total de execução será maior, pois nenhuma modalidade é interrompida.

Round Robin com semáforo de contagem permite execuções simultâneas, mas interrompe modalidades que ultrapassam o quantum, o que pode melhorar a eficiência em termos de utilização de recursos.

OBS.: O monitor não foi utilizado diretamente no código anterior porque, na prática, ele é bastante semelhante ao semáforo binário.